# A good decorator

```
def decorator_func(some_func):
  def wrapper(*args, **kwargs): // decorated func with *args and **kwargs
    result = some_func(*args, **kwargs)
    print('{}() returned type {}'.format(some_func.__name__, type(result)))
    return result
  return wrapper
  
@decorator_func
def some_func(value):
  return value
```

# Counter

- You can operate on function property inside the inner function: `func_name.prop += val`
- However, the propery initial value must be declared before the outer function's return call
- It can be accessed outside as: `func_name.prop`
- It works because function in python is also an object

```
from functools import wraps

def counter(func):
  @wraps(func) // You need this for safe docstring support
  def wrapper(*args, **kwargs):
    wrapper.count += 1 // increment a property
    return wrapper.count
  wrapper.count = 0 // declare the property

  return wrapper

@counter
def foo():
  print('calling foo()')
  
foo()
foo()

print('foo() was called {} times.'.format(foo.count))
```

# `@wraps(func)` in Decorators

```
@counter
def foo():
    """ Some Docstring. """
    print('calling foo()')
```

- When using `foo.__doc__` it will retrieve docstring of decorator, not the decorated function

To safeguard this, use `@wraps(func)` inside decorator above wrapper/inner function:
- It will show decorated function's docstring : `some_func.__doc__`
- It will show decorated function's name : `some_func.__name__`
- It will show decorated function's default parameters : `some_func.__defaults__`
- It will show the original function : `some_func.__wrapped__`
- It will call the original function : `some_func.__wrapped__(params)`
- Without it, these metadata cannot be accessed for Decorators

# A decorator factory

This is a process of creating a decorator function that takes in parameters. This is nothing but putting the decorator function inside another function that:
- takes the input for that decorator as parameter
- returns the decorator

```
def decorator_factory_func(n):
    """Define and return a decorator"""
    def decorator(func):
        def wrapper(*args, **kwargs):
            for i in range(n):
                func(*args, **kwargs)
            return wrapper
    return decorator

@decorator_factory_func(3) // calling the decorator factory function
def print_sum(a, b):
    print(a + b)   
```

Under the hood :
```
run_three_times = decorator_factory_func(3) // calling factory func and store decorator as result

@run_three_times // use that decorator
def print_sum(a, b):
    print(a + b)
```    

In short:

```    
@run_n_times(3)
def print_sum(a, b):
print(a + b)
```

# Modifying Builtin funcs with decorator factory

```
def run_n_times(n):
  """Define and return a decorator"""
  def decorator(func):
    def wrapper(*args, **kwargs):
      for i in range(n):
        func(*args, **kwargs) // repeat the occurence of input param func
    return wrapper
  return decorator

new_print = run_n_times(3)(print)

new_print('What is happening?!?!')  
```